In [ ]:
%load_ext autoreload

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s') # NOTSET, DEBUG, INFO, WARN, ERROR, CRITICAL

from JPAS_DA import global_setup

from JPAS_DA.data import loading_tools
from JPAS_DA.data import cleaning_tools
from JPAS_DA.data import crossmatch_tools
from JPAS_DA.data import process_dset_splits
from JPAS_DA.data import wrapper_data_loaders

from JPAS_DA.models import model_building_tools
from JPAS_DA.training import save_load_tools
from JPAS_DA.evaluation import evaluation_tools
from JPAS_DA.wrapper_wandb import wrapper_tools
from JPAS_DA.evaluation import evaluation_tools

import os
import torch
import numpy as np

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

from JPAS_DA.utils import plotting_utils
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('default')
plt.close('all')
font, rcnew = plotting_utils.matplotlib_default_config()
mpl.rc('font', **font)
plt.rcParams.update(rcnew)
plt.style.use('tableau-colorblind10')
%matplotlib widget

from JPAS_DA.utils import aux_tools
aux_tools.set_seed(42)

In [ ]:
path_load_no_DA = "09_no_DA"
path_load_DA = "09_DA"

In [ ]:
_, model_encoder_no_DA = save_load_tools.load_model_from_checkpoint(os.path.join(global_setup.path_models, path_load_no_DA, "model_encoder.pt"), model_building_tools.create_mlp)
_, model_downstream_no_DA = save_load_tools.load_model_from_checkpoint(os.path.join(global_setup.path_models, path_load_no_DA, "model_downstream.pt"), model_building_tools.create_mlp)

_, model_encoder_DA = save_load_tools.load_model_from_checkpoint(os.path.join(global_setup.path_models, path_load_DA, "model_encoder.pt"), model_building_tools.create_mlp)
_, model_downstream_DA = save_load_tools.load_model_from_checkpoint(os.path.join(global_setup.path_models, path_load_DA, "model_downstream.pt"), model_building_tools.create_mlp)

_ = evaluation_tools.compare_model_parameters(model_downstream_no_DA, model_downstream_DA, rtol=1e-2, atol=1e-2)

In [ ]:
_, config_no_DA = wrapper_tools.load_and_massage_config_file(os.path.join(global_setup.path_models, path_load_no_DA, "config.yaml"), path_load_no_DA)
_, config_DA = wrapper_tools.load_and_massage_config_file(os.path.join(global_setup.path_models, path_load_DA, "config.yaml"), path_load_DA)

In [ ]:
config_data = config_DA["data"]

tmp_key = "data_paths"
root_path = config_data[tmp_key]["root_path"]
load_JPAS_data = config_data[tmp_key]["load_JPAS_data"]
load_DESI_data = config_data[tmp_key]["load_DESI_data"]
random_seed_load = config_data[tmp_key]["random_seed_load"]

tmp_key = "dict_clean_data_options"
apply_masks = config_data[tmp_key]["apply_masks"]
mask_indices = config_data[tmp_key]["mask_indices"]
magic_numbers = config_data[tmp_key]["magic_numbers"]
i_band_sn_threshold = config_data[tmp_key]["i_band_sn_threshold"]
z_lim_QSO_cut = config_data[tmp_key]["z_lim_QSO_cut"]

tmp_key = "dict_split_data_options"
train_ratio_both = config_data[tmp_key]["train_ratio_both"]
val_ratio_both = config_data[tmp_key]["val_ratio_both"]
test_ratio_both = config_data[tmp_key]["test_ratio_both"]
random_seed_split_both = config_data[tmp_key]["random_seed_split_both"]
train_ratio_only_DESI = config_data[tmp_key]["train_ratio_only_DESI"]
val_ratio_only_DESI = config_data[tmp_key]["val_ratio_only_DESI"]
test_ratio_only_DESI = config_data[tmp_key]["test_ratio_only_DESI"]
random_seed_split_only_DESI = config_data[tmp_key]["random_seed_split_only_DESI"]

define_dataset_loaders_keys = ['DESI_only', "JPAS_matched"]
keys_xx = config_data["features_labels_options"]["keys_xx"]
keys_yy = ["SPECTYPE_int", "TARGETID", "DESI_FLUX_R"]
normalize = True
provided_normalization = config_data["provided_normalization"]

In [ ]:
# ───────────────────────────────────────────────────── #
# 1. Load raw JPAS and DESI datasets
# ───────────────────────────────────────────────────── #
logging.info("\n\n1️⃣: Loading datasets from disk...")
DATA = loading_tools.load_dsets(
    root_path=root_path,
    datasets_jpas=load_JPAS_data,
    datasets_desi=load_DESI_data,
    random_seed=random_seed_load
)

# ───────────────────────────────────────────────────── #
# 2. Apply cleaning and masking procedures
# ───────────────────────────────────────────────────── #
logging.info("\n\n2️⃣: Cleaning and masking data...")
DATA = cleaning_tools.clean_and_mask_data(
    DATA=DATA,
    apply_masks=apply_masks,
    mask_indices=mask_indices,
    magic_numbers=magic_numbers,
    i_band_sn_threshold=i_band_sn_threshold,
    z_lim_QSO_cut=z_lim_QSO_cut
)

# ───────────────────────────────────────────────────── #
# 3. Crossmatch JPAS and DESI using TARGETID
# ───────────────────────────────────────────────────── #
logging.info("\n\n3️⃣: Crossmatching JPAS and DESI TARGETIDs...")
Dict_LoA = {"both": {}, "only": {}}
IDs_only_DESI, IDs_only_JPAS, IDs_both, \
Dict_LoA["only"]["DESI"], Dict_LoA["only"]["JPAS"], \
Dict_LoA["both"]["DESI"], Dict_LoA["both"]["JPAS"] = crossmatch_tools.crossmatch_IDs_two_datasets(
    DATA["DESI"]['TARGETID'], DATA["JPAS"]['TARGETID']
)

# ───────────────────────────────────────────────────── #
# 4. Perform train/val/test splits
# ───────────────────────────────────────────────────── #
logging.info("\n\n4️⃣: Splitting data into train/val/test...")
Dict_LoA_split = {"both": {}, "only": {}}

Dict_LoA_split["both"]["JPAS"] = process_dset_splits.split_LoA(
    Dict_LoA["both"]["JPAS"], train_ratio_both, val_ratio_both, test_ratio_both, seed=random_seed_split_both
)
Dict_LoA_split["both"]["DESI"] = process_dset_splits.split_LoA(
    Dict_LoA["both"]["DESI"], train_ratio_both, val_ratio_both, test_ratio_both, seed=random_seed_split_both
)
Dict_LoA_split["only"]["DESI"] = process_dset_splits.split_LoA(
    Dict_LoA["only"]["DESI"], train_ratio_only_DESI, val_ratio_only_DESI, test_ratio_only_DESI, seed=random_seed_split_only_DESI
)

# ───────────────────────────────────────────────────── #
# 5. Load data
# ───────────────────────────────────────────────────── #
logging.info("\n\n5️⃣: Load and normalize data...")

xx_dict = {}
yy_dict = {}
for key_dset in ["val", "test"]:
    xx_dict[key_dset] = {}
    yy_dict[key_dset] = {}
    logging.info(f"⚙️ Preparing split: {key_dset}")
    for key_loader in define_dataset_loaders_keys:
        logging.info(f"├── {key_loader}")
        if key_loader == "DESI_combined":
            LoA, xx, yy = process_dset_splits.extract_and_combine_DESI_data(
                Dict_LoA_split["only"]["DESI"][key_dset], Dict_LoA_split["both"]["DESI"][key_dset], DATA["DESI"], keys_xx, keys_yy
            )
        elif key_loader == "DESI_only":
            LoA, xx, yy = process_dset_splits.extract_data_using_LoA(
                Dict_LoA_split["only"]["DESI"][key_dset], DATA["DESI"], keys_xx, keys_yy
            )
        elif key_loader == "DESI_matched":
            LoA, xx, yy = process_dset_splits.extract_data_using_LoA(
                Dict_LoA_split["both"]["DESI"][key_dset], DATA["DESI"], keys_xx, keys_yy
            )
        elif key_loader == "JPAS_matched":
            LoA, xx, yy = process_dset_splits.extract_data_using_LoA(
                Dict_LoA_split["both"]["JPAS"][key_dset], DATA["JPAS"], keys_xx, keys_yy
            )
        # Normalize, reshape, and stack all features in one pass
        xx_stacked = np.concatenate([
            np.atleast_2d((xx[kk] - provided_normalization[0][ii]) / provided_normalization[1][ii]).reshape(xx[kk].shape[0], -1)
            for ii, kk in enumerate(xx)
        ], axis=1)

        # Store as torch tensor
        xx_dict[key_dset][key_loader] = torch.tensor(xx_stacked, dtype=torch.float32, device="cpu")
        yy_dict[key_dset][key_loader] = yy


In [ ]:
# compute the validation set results for the networks trained without domain adaptation (employing the DESI mock spectra)
key_dset = "val"
key_loader = "DESI_only"

tmp_xx = xx_dict[key_dset][key_loader]
with torch.no_grad():
    tmp_features = model_encoder_no_DA(tmp_xx)
    tmp_logits = model_downstream_no_DA(tmp_features)
tmp_yy_pred_P = torch.nn.functional.softmax(tmp_logits, dim=1).cpu().numpy()

yy_dict[key_dset][key_loader]['no_DA_features'] = tmp_features.cpu().numpy()
yy_dict[key_dset][key_loader]['no_DA_pred_Probabilities'] = tmp_yy_pred_P
yy_dict[key_dset][key_loader]['no_DA_pred_labels'] = np.argmax(tmp_yy_pred_P, axis=1)

# # compute the validation set results for the networks trained with domain adaptation (employing the JPAS spectra)
# key_dset = "val"
# key_loader = "JPAS_matched"

# tmp_xx = xx_dict[key_dset][key_loader]
# with torch.no_grad():
#     tmp_features = model_encoder_DA(tmp_xx)
#     tmp_logits = model_downstream_DA(tmp_features)
# tmp_yy_pred_P = torch.nn.functional.softmax(tmp_logits, dim=1).cpu().numpy()

# yy_dict[key_dset][key_loader]['DA_features'] = tmp_features.cpu().numpy()
# yy_dict[key_dset][key_loader]['DA_pred_Probabilities'] = tmp_yy_pred_P
# yy_dict[key_dset][key_loader]['DA_pred_labels'] = np.argmax(tmp_yy_pred_P, axis=1)

# compute the test set results for the networks trained without domain adaptation (employing the JPAS spectra)
key_dset = "test"
key_loader = "JPAS_matched"

tmp_xx = xx_dict[key_dset][key_loader]
with torch.no_grad():
    tmp_features = model_encoder_no_DA(tmp_xx)
    tmp_logits = model_downstream_no_DA(tmp_features)
tmp_yy_pred_P = torch.nn.functional.softmax(tmp_logits, dim=1).cpu().numpy()

yy_dict[key_dset][key_loader]['no_DA_features'] = tmp_features.cpu().numpy()
yy_dict[key_dset][key_loader]['no_DA_pred_Probabilities'] = tmp_yy_pred_P
yy_dict[key_dset][key_loader]['no_DA_pred_labels'] = np.argmax(tmp_yy_pred_P, axis=1)

# compute the test set results for the networks trained with domain adaptation (employing the JPAS spectra)
key_dset = "test"
key_loader = "JPAS_matched"

tmp_xx = xx_dict[key_dset][key_loader]
with torch.no_grad():
    tmp_features = model_encoder_DA(tmp_xx)
    tmp_logits = model_downstream_DA(tmp_features)
tmp_yy_pred_P = torch.nn.functional.softmax(tmp_logits, dim=1).cpu().numpy()

yy_dict[key_dset][key_loader]['DA_features'] = tmp_features.cpu().numpy()
yy_dict[key_dset][key_loader]['DA_pred_Probabilities'] = tmp_yy_pred_P
yy_dict[key_dset][key_loader]['DA_pred_labels'] = np.argmax(tmp_yy_pred_P, axis=1)

In [ ]:
key_dset = "val"
key_loader = "DESI_only"
yy_true_no_DA_val = yy_dict[key_dset][key_loader]['SPECTYPE_int']
yy_pred_P_no_DA_val = yy_dict[key_dset][key_loader]['no_DA_pred_Probabilities']
yy_pred_no_DA_val = yy_dict[key_dset][key_loader]['no_DA_pred_labels']
confusion_matrix = evaluation_tools.plot_confusion_matrix(
    yy_true_no_DA_val, yy_pred_P_no_DA_val,
    class_names=global_setup.class_names,
    cmap=plt.cm.RdYlGn, title="Validation no DA"
)

# key_dset = "val"
# key_loader = "JPAS_matched"
# yy_true_DA_val = yy_dict[key_dset][key_loader]['SPECTYPE_int']
# yy_pred_P_DA_val = yy_dict[key_dset][key_loader]['DA_pred_Probabilities']
# yy_pred_DA_val = yy_dict[key_dset][key_loader]['DA_pred_labels']
# confusion_matrix = evaluation_tools.plot_confusion_matrix(
#     yy_true_DA_val, yy_pred_P_DA_val,
#     class_names=global_setup.class_names,
#     cmap=plt.cm.RdYlGn, title="Validation DA"
# )

key_dset = "test"
key_loader = "JPAS_matched"
yy_true_no_DA_test = yy_dict[key_dset][key_loader]['SPECTYPE_int']
yy_pred_P_no_DA_test = yy_dict[key_dset][key_loader]['no_DA_pred_Probabilities']
yy_pred_no_DA_test = yy_dict[key_dset][key_loader]['no_DA_pred_labels']
confusion_matrix = evaluation_tools.plot_confusion_matrix(
    yy_true_no_DA_test, yy_pred_P_no_DA_test,
    class_names=global_setup.class_names,
    cmap=plt.cm.RdYlGn, title="Test no DA"
)

key_dset = "test"
key_loader = "JPAS_matched"
yy_true_DA_test = yy_dict[key_dset][key_loader]['SPECTYPE_int']
yy_pred_P_DA_test = yy_dict[key_dset][key_loader]['DA_pred_Probabilities']
yy_pred_DA_test = yy_dict[key_dset][key_loader]['DA_pred_labels']
confusion_matrix = evaluation_tools.plot_confusion_matrix(
    yy_true_DA_test, yy_pred_P_DA_test,
    class_names=global_setup.class_names,
    cmap=plt.cm.RdYlGn, title="Test DA"
)

In [ ]:
evaluation_tools.compare_TPR_confusion_matrices(
    yy_true_no_DA_val,
    yy_pred_P_no_DA_val,
    yy_true_no_DA_test,
    yy_pred_P_no_DA_test,
    class_names=global_setup.class_names,
    figsize=(10, 7),
    cmap='seismic',
    title='Performance lost no DA -- Val. (DESI-mocks) VS test (JPAS spectra)',
    name_1 = "Val. Mock",
    name_2 = "Test JPAS"
)
metrics = evaluation_tools.compare_sets_performance(
    yy_true_no_DA_val, yy_pred_P_no_DA_val,
    yy_true_no_DA_test, yy_pred_P_no_DA_test,
    class_names=global_setup.class_names,
    name_1="Val. Mock",
    name_2="Test JPAS"
)

# evaluation_tools.compare_TPR_confusion_matrices(
#     yy_true_DA_val,
#     yy_pred_P_DA_val,
#     yy_true_DA_test,
#     yy_pred_P_DA_test,
#     class_names=global_setup.class_names,
#     figsize=(10, 7),
#     cmap='seismic',
#     title='Performance lost with DA -- Val. (JPAS spectra) VS test (JPAS spectra)',
#     name_1 = "Val. JPAS",
#     name_2 = "Test JPAS"
# )
# metrics = evaluation_tools.compare_sets_performance(
#     yy_true_DA_val, yy_pred_P_DA_val,
#     yy_true_DA_test, yy_pred_P_DA_test,
#     class_names=global_setup.class_names,
#     name_1="Val. JPAS",
#     name_2="Test JPAS"
# )

evaluation_tools.compare_TPR_confusion_matrices(
    yy_true_no_DA_test,
    yy_pred_P_no_DA_test,
    yy_true_DA_test,
    yy_pred_P_DA_test,
    class_names=global_setup.class_names,
    figsize=(10, 7),
    cmap='seismic',
    title='Performance comparison no DA VES with DA (Test sets JPAS spectra)',
    name_1 = "No DA",
    name_2 = "With DA"
)
metrics = evaluation_tools.compare_sets_performance(
    yy_true_no_DA_test, yy_pred_P_no_DA_test,
    yy_true_DA_test, yy_pred_P_DA_test,
    class_names=global_setup.class_names,
    name_1="No DA",
    name_2="With DA"
)

In [ ]:
# Prepare magnitudes for combinations of interest
key_pairs = [("val", "JPAS_matched"), ("test", "JPAS_matched"), ("val", "DESI_only"), ("test", "DESI_only")]
mag_dict = {}
for key_dset, key_loader in key_pairs:
    flux_R = yy_dict[key_dset][key_loader]['DESI_FLUX_R']
    magnitude_R = np.full_like(flux_R, np.nan)
    valid_flux = flux_R > 0
    magnitude_R[valid_flux] = 22.5 - 2.5 * np.log10(flux_R[valid_flux])
    mag_dict[(key_dset, key_loader)] = magnitude_R

# Compute global range from all sets
all_mags = np.concatenate([v[np.isfinite(v)] for v in mag_dict.values()])
min_mag, max_mag = np.nanmin(all_mags), np.nanmax(all_mags)
ranges = [(min_mag, 17), (17, 19), (19, 21), (21, 22), (22, 22.5), (22.5, max_mag)]
colors = ['k', 'purple', 'blue', 'green', 'orange', 'red']

masks_all = plotting_utils.plot_histogram_with_ranges_multiple(
    mag_dict, ranges=ranges, colors=colors, bins=42,
    x_label="DESI Magnitude (R)",
    title="DESI R-band Magnitudes by Dataset Split and Loader"
)

# massage masks_all to a dictionary with mask like bin indices
bin_index_dict = {}
for key in masks_all.keys():
    n_samples = len(next(iter(masks_all[key].values())))  # length from first mask
    bin_indices = np.full(n_samples, -1, dtype=int)  # default: -1 means "unassigned"
    for bin_id, mag_range in enumerate(ranges):
        mask = masks_all[key][mag_range]
        bin_indices[mask] = bin_id
    bin_index_dict[key] = bin_indices

# include this mask as a new feature in the yy_dicts
for key in bin_index_dict:
    key_dset, key_loader = key
    yy_dict[key_dset][key_loader]['MAG_BIN_ID'] = bin_index_dict[key]
